In [1]:
import pandas as pd
from recommenders.datasets import movielens

# 加载MovieLens数据
data = movielens.load_pandas_df(
    size="100k",
    header=["userID", "itemID", "rating"],
    genres_col="genre"
)

print(data.head())

100%|████████████████████████████████████████████████████████████████████████████| 4.81k/4.81k [00:01<00:00, 3.30kKB/s]


   userID  itemID  rating                             genre
0     196     242     3.0                            Comedy
1     186     302     3.0  Crime|Film-Noir|Mystery|Thriller
2      22     377     1.0                 Children's|Comedy
3     244      51     2.0         Drama|Romance|War|Western
4     166     346     1.0                       Crime|Drama


In [2]:
import sklearn.preprocessing

# 将电影类型编码为多热向量
genres_encoder = sklearn.preprocessing.MultiLabelBinarizer()
data["genre"] = genres_encoder.fit_transform(
    data["genre"].apply(lambda s: s.split("|"))
).tolist()

print("电影类型:", genres_encoder.classes_)
print(data.head())

电影类型: ['Action' 'Adventure' 'Animation' "Children's" 'Comedy' 'Crime'
 'Documentary' 'Drama' 'Fantasy' 'Film-Noir' 'Horror' 'Musical' 'Mystery'
 'Romance' 'Sci-Fi' 'Thriller' 'War' 'Western' 'unknown']
   userID  itemID  rating                                              genre
0     196     242     3.0  [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1     186     302     3.0  [0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, ...
2      22     377     1.0  [0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3     244      51     2.0  [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, ...
4     166     346     1.0  [0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...


In [3]:
from recommenders.datasets.python_splitters import python_random_split

train, test = python_random_split(data, ratio=0.75, seed=42)
print(f"{len(train)} 条训练样本和 {len(test)} 条测试样本")

75000 条训练样本和 25000 条测试样本


In [4]:
def build_wide_features():
    crossed_feature = tf.feature_column.crossed_column(
        ["userID", "itemID"], hash_bucket_size=1000)
    return [crossed_feature]
def build_deep_features(user_dim, item_dim):
    user_col = tf.feature_column.categorical_column_with_identity("userID", num_buckets=1000)
    item_col = tf.feature_column.categorical_column_with_identity("itemID", num_buckets=2000)
    
    user_emb = tf.feature_column.embedding_column(user_col, dimension=user_dim)
    item_emb = tf.feature_column.embedding_column(item_col, dimension=item_dim)
    
    genre_indicator = tf.feature_column.numeric_column("genre", shape=(19,))
    
    return [user_emb, item_emb, genre_indicator]

In [15]:
import tensorflow as tf
tf.compat.v1.disable_v2_behavior()
model = tf.estimator.DNNLinearCombinedRegressor(
    model_dir=model_dir,
    
    linear_feature_columns=build_wide_features(),
    
    dnn_feature_columns=build_deep_features(DNN_USER_DIM, DNN_ITEM_DIM),
    
    dnn_hidden_units=DNN_HIDDEN_UNITS,
    
    dnn_optimizer=tf.compat.v1.train.AdagradOptimizer(learning_rate=DNN_OPTIMIZER_LR),
    linear_optimizer=tf.compat.v1.train.FtrlOptimizer(learning_rate=LINEAR_OPTIMIZER_LR),
    dnn_dropout=DNN_DROPOUT,
    batch_norm=bool(DNN_BATCH_NORM),
)

AttributeError: module 'tensorflow' has no attribute 'estimator'

In [13]:
# 定义输入函数
def input_fn(data, batch_size):
    return tf.compat.v1.estimator.inputs.pandas_input_fn(
        x=data,
        y=data["rating"],
        batch_size=batch_size,
        num_epochs=None,
        shuffle=True,
    )

# 训练模型
model.train(input_fn=input_fn(train, BATCH_SIZE), steps=STEPS)

# 评估模型
results = model.evaluate(input_fn=input_fn(test, BATCH_SIZE), steps=None)
print("测试集上的RMSE:", results["average_loss"]**0.5)

NameError: name 'model' is not defined

In [7]:
class EvalRecordHook(tf.estimator.SessionRunHook):
    def __init__(self, log_path, every_n_iter):
        self.log_path = log_path
        self.every_n_iter = every_n_iter
        self.records = []

    def after_run(self, run_context, run_values):
        if run_context.session.run(tf.compat.v1.train.get_global_step()) % self.every_n_iter == 0:
            eval_results = model.evaluate(input_fn=input_fn(test, BATCH_SIZE), steps=None)
            self.records.append({
                "step": run_context.session.run(tf.compat.v1.train.get_global_step()),
                "rmse": eval_results["average_loss"]**0.5
            })
            with open(self.log_path, "w") as f:
                json.dump(self.records, f)

eval_hook = EvalRecordHook(os.path.join(model_dir, "eval_records.json"), 1000)
model.train(input_fn=input_fn(train, BATCH_SIZE), steps=STEPS, hooks=[eval_hook])

import matplotlib.pyplot as plt
import json

with open(os.path.join(model_dir, "eval_records.json"), "r") as f:
    records = json.load(f)

steps = [record["step"] for record in records]
rmse = [record["rmse"] for record in records]

plt.figure(figsize=(10, 6))
plt.plot(steps, rmse)
plt.title("模型训练过程中RMSE的变化")
plt.xlabel("训练步数")
plt.ylabel("RMSE")
plt.show()

SyntaxError: invalid syntax (3088309089.py, line 19)

In [8]:
from recommenders.evaluation.python_evaluation import (
    rmse, mae, ndcg_at_k, precision_at_k
)

# 获取测试集上的预测结果
predictions = model.predict(input_fn=input_fn(test, BATCH_SIZE))
pred_ratings = [p['predictions'][0] for p in predictions]

# 计算各种评估指标
print("RMSE:", rmse(test["rating"], pred_ratings))
print("MAE:", mae(test["rating"], pred_ratings))
print("NDCG@10:", ndcg_at_k(test, pred_ratings, k=10))
print("Precision@10:", precision_at_k(test, pred_ratings, k=10))

NameError: name 'model' is not defined

In [9]:
def recommend_movies(user_id, top_k=10):
    # 获取所有电影ID
    all_movies = data["itemID"].unique()
    
    # 构造用户-电影对
    user_movie_pairs = pd.DataFrame({
        "userID": [user_id] * len(all_movies),
        "itemID": all_movies,
        "genre": data[data["itemID"].isin(all_movies)]["genre"].tolist()
    })
    
    # 使用模型预测评分
    predictions = model.predict(input_fn=input_fn(user_movie_pairs, BATCH_SIZE))
    pred_ratings = [p['predictions'][0] for p in predictions]
    
    # 将预测结果与电影ID配对并排序
    movie_ratings = list(zip(all_movies, pred_ratings))
    top_movies = sorted(movie_ratings, key=lambda x: x[1], reverse=True)[:top_k]
    
    return top_movies

# 为用户196推荐10部电影
recommendations = recommend_movies(196, top_k=10)
print("为用户196推荐的电影:")
for movie, rating in recommendations:
    print(f"电影ID: {movie}, 预测评分: {rating:.2f}")

ValueError: All arrays must be of the same length

In [10]:
def analyze_recommendations(user_id, recommendations):
    user_history = data[data["userID"] == user_id]
    print(f"用户{user_id}的历史评分:")
    print(user_history[["itemID", "rating"]].to_string(index=False))
    
    print("\n推荐电影:")
    for movie, rating in recommendations:
        if movie in user_history["itemID"].values:
            actual_rating = user_history[user_history["itemID"] == movie]["rating"].values[0]
            print(f"电影ID: {movie}, 预测评分: {rating:.2f}, 实际评分: {actual_rating}")
        else:
            print(f"电影ID: {movie}, 预测评分: {rating:.2f}, 新推荐")

analyze_recommendations(196, recommendations)

NameError: name 'recommendations' is not defined

2.18.0


AttributeError: module 'tensorflow._api.v2.compat.v1' has no attribute 'estimator'